<a href="https://colab.research.google.com/github/RaminVali/CL_WebScraping_ML_Prediction/blob/main/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting up Selenium Chorme dirver to use for scraping and scrolling donwn etc.

Google colab has issues with selenium and the associated websidrver because their ubunto release does not include snap. Here is a workaround for this issue on google colab. Please see [here](https://stackoverflow.com/questions/75164313/selenium-in-google-colab-stopped-working-showing-an-error-as-service-chromedrive) for more details.

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get install chromium chromium-driver
apt-get update


# Install selenium
pip install selenium

# Install
pip install webdriver-manager

Executing: /tmp/apt-key-gpghome.5gvaEdEssx/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.9Yw062pkLR/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.YgvOEFbZTg/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [12]:
! ls /usr/bin/chromedriver

/usr/bin/chromedriver


Importing Packages

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
import os
from bs4 import BeautifulSoup
from urllib.parse import urljoin

import requests
import pandas as pd

## Setting up the webdriver with appropriate options
To not be flagged as a robot by the website, see [here](https://stackoverflow.com/questions/71885891/urllib3-exceptions-maxretryerror-httpconnectionpoolhost-localhost-port-5958)

In [3]:


#service = Service(executable_path='/usr/lib/chromium-browser/chromedriver/usr/bin') #this gives a version mismatch error bweteen the driver and the Chormium

def get_driver():
  service = Service(executable_path='/usr/bin/chromedriver') #this is where the correct version of the chromium and webdriver live


  options = Options()
  options.add_argument("--headless")
  options.add_argument("--no-sandbox")
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('--disable-blink-features=AutomationControlled')

  driver = webdriver.Chrome(service = service, options = options)
  return driver


Setting up the driver and doing infinite scrolling to get all the html form the main website page. We will get the hrefs with all the links. [Source](https://medium.com/analytics-vidhya/using-python-and-selenium-to-scrape-infinite-scroll-web-pages-825d12c24ec7)

In [ ]:
# Web scrapper for infinite scrolling page
driver = get_driver()
driver.get("https://www.privateislandsonline.com/search?availability=sale&region=&q2=&price_range=0%3A50000000&size_range=0%3A1000")
time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 2 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break



soup = BeautifulSoup(driver.page_source, "html.parser")


Saving the contents we scraped

In [ ]:
with open("output1.html", "w", encoding='utf-8') as file:
    file.write(str(soup))

Loading the saved contents again

In [16]:
path = '/content/drive/MyDrive/Colab Notebooks/Private_Island_Project/Island_Project/output1.html'

with open(path, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')

##### Extract Individual Island URLs #####
urls = []
for parent in soup.find_all('div', class_="img-container"):
    url = parent.find('a').get('href')
    urls.append(url)

print(f"We have found {len(urls)} islands for sale")

We have found 611 islands for sale


We will go through each 611 URLs and dump the page contents into a seperate folder. The folder name is going to be from the URL, where it is region/country/island_name or region/region/country/island_name.

test_url = ['https://www.privateislandsonline.com/asia/thailand/rangyai-island',
 'https://www.privateislandsonline.com/europe/spain/2867']

In [4]:
test_url = urls[0:2]
test_url

NameError: ignored

In [5]:
test_url = ['https://www.privateislandsonline.com/asia/thailand/rangyai-island',
 'https://www.privateislandsonline.com/europe/spain/2867']

In [6]:
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Private_Island_Project/Island_Project/Data"


In [8]:
  Driver = get_driver()
  Driver.get(test_url[1])

In [15]:
Driver.title
Driver.current_url
from selenium.webdriver.common.by import By

basic_info = Driver.find_element(By.ID,"island-details")


In [ ]:
print(basic_info.text)

In [ ]:
basic_info = Driver.find_element(By.CLASS_NAME,"basic-info.mb-50") # you need a "." between basic-info and mb-50 because "selector"
print(basic_info.text)

In [25]:
other_info = Driver.find_element(By.CLASS_NAME,"hidden-xs.descriptionContainer")
print(other_info.text)

Large turnkey private island available for sale in Spain with luxurious amenities, excellent location and easy access by boat or helicopter.
What is a blackbook Listing?
Blackbook private island listings are available to the public, without compromising the privacy requirements of our sellers. Some clients prefer not to make the details of their private island sale known. These may include celebrities, development firms or individuals simply concerned with privacy.


In [26]:
  Driver.get(test_url[0])

In [28]:
Driver.title
Driver.current_url


'https://www.privateislandsonline.com/asia/thailand/rangyai-island'

In [39]:
imgs = Driver.find_elements("xpath","//img")
for img in imgs:
    print(img.get_attribute("src"))

https://www.privateislandsonline.com/img/front/new-logo.png
https://www.privateislandsonline.com/img/front/logo_small.png
https://www.privateislandsonline.com/img/front/navSocialIcon_fb.png
https://www.privateislandsonline.com/img/front/navSocialIcon_tw.png
https://www.privateislandsonline.com/img/front/navSocialIcon_gp.png
https://www.google.com/images/cleardot.gif
https://www.google.com/images/cleardot.gif
https://www.google.com/images/cleardot.gif
https://www.privateislandsonline.com/images/mappoint.png
https://www.privateislandsonline.com/uploads/resize/_597_image_fe02c338d8.jpg-1360-1100.jpg
https://www.privateislandsonline.com/uploads/resize/_597_image_25bce99231.jpg-1360-1100.jpg
https://www.privateislandsonline.com/uploads/resize/_597_image_d24a8525fc.jpg-1360-1100.jpg
https://www.privateislandsonline.com/uploads/resize/_597_image_e971b277e1.jpg-1360-1100.jpg
https://www.privateislandsonline.com/uploads/resize/_597_image_fe02c338d8.jpg-100-64.jpg
https://www.privateislandsonlin

In [35]:
image

[<selenium.webdriver.remote.webelement.WebElement (session="3b29ac8953a83a6c4f2bb90ef5824df4", element="3f77eba8-5335-4c20-8276-8e9cc8f05a0a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3b29ac8953a83a6c4f2bb90ef5824df4", element="841f3803-8cc1-4fa4-b2fc-2779c13bbc9f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3b29ac8953a83a6c4f2bb90ef5824df4", element="0ac41d60-175b-49a7-ae8e-ec9180151e80")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3b29ac8953a83a6c4f2bb90ef5824df4", element="89b36d82-6a4a-4e2a-a31e-f87631b7776b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3b29ac8953a83a6c4f2bb90ef5824df4", element="ff1ebb03-bd85-4ff7-822d-9c8d40682fc2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3b29ac8953a83a6c4f2bb90ef5824df4", element="a11528cc-0b82-497e-98c6-5034a583c683")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3b29ac8953a83a6c4f2bb90ef5824df4", element="0ed8cd38-8edf-4eb7-9cbd-47

In [30]:
def get_html(url, sleep = 5, retries = 3):
  time.sleep(sleep)
  soup = None
  Driver = get_driver()
  for i in range (1, retries+1):
    try:
      time.sleep(sleep)  # Allow 2 seconds for the web page to open
      Driver.get(url)

    except TimeoutException as ex:
        isrunning = 0
        print(f"Timeout error on {url} the exception is: {ex}")
        continue
    else:
        break
  return soup

def make_path(url):
  # make file name
  part = url.split("/")
  if len(part) == 6:
    file_name = part[-3]+'-'+part[-2]+'-'+part[-1]
  else:
    file_name = part[-4]+'-'+part[-3]+'-'+part[-2]+'-'+part[-1]

  save_path = os.path.join(DATA_DIR,file_name)
  return save_path, file_name

def save_html(all_html, save_path):
  with open(save_path+".html", "w+", encoding='utf-8') as file:
    file.write(str(all_html))


In [31]:
for url in test_url:

  (save_path, file_name) = make_path(url)
  if os.path.exists(save_path):
    continue

  get_html(url)

  save_html(soup, save_path)




In [32]:
islands_files = os.listdir(DATA_DIR)
islands_files = [i for i in islands_files if '.html' in i]

In [33]:
islands_files

['asia-thailand-rangyai-island.html', 'europe-spain-2867.html']

In [ ]:
with open(os.path.join(DATA_DIR,islands_files[1]), 'r') as f:
  html = f.read()

soup = BeautifulSoup(html)
basic_info = soup.select("div", {'ul':'class="basic-info mb-50"'})
basic_info

In [130]:
print(basic_info)

None


In [141]:
content = soup.find('basic-info mb-50')

print(content)

None


In [104]:
df = pd.read_html(str(soup), attrs = {'class': 'basic-info mb-50'})[0]

ValueError: ignored

In [ ]:
basic_info

In [25]:
response = requests.get(url)

In [26]:
response.content

b'<!DOCTYPE html>\n<html><head><meta charset="utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="viewport" content="width=device-width, initial-scale=1.0" /><meta name="author" content=""><title>Rangyai Island - Thailand, Asia   - Private Islands for Sale</title><meta name="og:title" content="Rangyai Island - Thailand, Asia   - Private Islands"><meta name="description" content="20 minutes east of the tourist friendly island of Phuket this 110 acre island features white sand..."><meta name="og:url" content="http%3A%2F%2Fwww.privateislandsonline.com%2Fasia%2Fthailand%2Frangyai-island"><meta property="og:image" content="http://www.privateislandsonline.com/uploads/597/image_fe02c338d8.jpg" /><meta name="keywords" content="Private Islands Inc., Private Islands Online, Private Islands Magazine, Private Island Appraisals, Private Island Broker, island for sale, private islands for sale, islands, island, sale, rent, private island, real estate, islands for sale, private is

In [86]:
id = 'island-details'

In [ ]:
html